In [1]:
import pandas as pd
import re
import pickle
import tfidf_matcher as tm
pd.options.mode.chained_assignment = None  # default='warn'
from urllib.request import Request, urlopen

In [2]:
with open('../Data/news.pkl', 'rb') as f:
    news = pickle.load(f)
    # 3844 unique authors

muckrack = pd.read_csv('../Data/muckrack_persons_fetchlist.csv')

In [3]:
def form_urls():
    temp = news[['author', 'article_count']].groupby('author').count().sort_values(['article_count'], ascending = False)
    temp = temp.reset_index()
    urls = []
    for i in range(len(temp)):
        url = temp.loc[i].author.lower()
        url = re.sub(r'\|.*$', '', url) # remove everything after |
        url = re.sub(r'\,.*$', '', url) # remove evrything after ,
        
        punctuations = '''!()[]{};:'"\,<>./?#$%^&*_~'''
        for x in url.lower(): 
            if x in punctuations: url = url.replace(x, "") # remove punctuations except hyphen
        url = re.sub(r'(\d+)', '', url) # remove numbers
        url = re.sub(r'(--)', '', url)
        url = re.sub(r'(\w+@\w+)', '', url) # remove emails
        url = ' '.join([i.strip() for i in url.split()]) # remove spaces and lowercase
        url = re.sub(r'(\s+)(and)(\s+)(\w*)(\s*)(\w*)', '', url) # remove anything after and
        keywords = ['sa', 'tulsa', 'editor', 'writer', 'world-herald', 'news', 'world', 'richmond', 'times-dispatch', 'new',
                   'hampshire', 'union', 'leader', 'for', 'the', 'state', 'journal', 'correspondant', 'sfgate', 'special',
                   'from', 'the', 'gazette', 'times', 'staff', 'senior', 'dr', 'correspondent', 'by', 'editorial board',
                   'research', 'wire reports', 'security', 'real', 'estate', 'to', 'post', 'and', 'courier', 'policy',
                   'commercial', 'bureau', 'political', 'roanoke', 'college', 'football', 'editorial','democrat-gazette',
                   'arizona', 'daily', 'star', '--hamburg', 'column', 'lincoln', 'managing', 'backstage', 'with', 'sports',
                   'ii', 'iii', 'capitol', 'media', 'services']
        url = ' '.join([i for i in url.split() if i not in keywords]) # remove other keywords
        url = re.sub(r'(@)', '', url) # remove handles
        url = 'https://muckrack.com/' + url.replace(' ', '-').strip('-') # make URL
        urls.append(url)
    temp['request_url'] = urls
    temp['redirect_url'] = urls
    temp['method'] = 'none'
    temp['confidence'] = '-'
    return temp

In [4]:
data = form_urls()

In [5]:
data

,author,article_count,request_url,redirect_url,method,confidence
0,Neil Shaw,523,https://muckrack.com/neil-shaw,https://muckrack.com/neil-shaw,none,-
1,James Rodger,512,https://muckrack.com/james-rodger,https://muckrack.com/james-rodger,none,-
2,Jack Davis,251,https://muckrack.com/jack-davis,https://muckrack.com/jack-davis,none,-
3,Adam Wells,223,https://muckrack.com/adam-wells,https://muckrack.com/adam-wells,none,-
4,Sophie McCoid,218,https://muckrack.com/sophie-mccoid,https://muckrack.com/sophie-mccoid,none,-
...,...,...,...,...,...,...
3833,Levi Sumagaysay,10,https://muckrack.com/levi-sumagaysay,https://muckrack.com/levi-sumagaysay,none,-
3834,Ben Ashford,10,https://muckrack.com/ben-ashford,https://muckrack.com/ben-ashford,none,-
3835,Craig Brown,10,https://muckrack.com/craig-brown,https://muckrack.com/craig-brown,none,-
3836,Courtney Connley,10,https://muckrack.com/courtney-connley,https://muckrack.com/courtney-connley,none,-


In [6]:
lookup = list(muckrack.url)
k_matches = 1
ngram_length = 3
def match_with_tfdif(wrong_url):
    wrong_url = [wrong_url]
    results = tm.matcher(wrong_url, lookup, k_matches, ngram_length)
    match_confidence = results['Match Confidence'].iloc[0]
    right_url = results['Lookup 1'].iloc[0]
    return match_confidence, right_url

In [7]:
match_with_tfdif('https://muckrack.com/ryan-taylor')

(0.89, 'https://muckrack.com/ryan-taylor-3')

In [ ]:
for j in range(len(data)):
    url = data.iloc[j].request_url
    try:
        req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
        res = urlopen(req)
        sauce = res.read()
        data.loc[j, 'method'] = 'brute force'
        data.loc[j, 'redirect_url'] = res.url
        print(j, "found brute force", url)
    except:
        confidence, url = match_with_tfdif(url)
        try:
            req = Request(url, headers={'User-Agent': 'Mozilla/5.0'})
            res = urlopen(req)
            sauce = res.read()
            data.loc[j, 'method'] = 'tfidf match'
            data.loc[j, 'confidence'] = confidence
            data.loc[j, 'request_url'] = url
            data.loc[j, 'redirect_url'] = res.url
            print(j, "found tfidf", url)
        except:
            print(j, "found nothing", url)
            data.loc[j, 'method'] = 'no match'
            # google scraper
    print()

0 found brute force https://muckrack.com/neil-shaw

1 found brute force https://muckrack.com/james-rodger

2 found brute force https://muckrack.com/jack-davis

3 found brute force https://muckrack.com/adam-wells

4 found brute force https://muckrack.com/sophie-mccoid

5 found brute force https://muckrack.com/jack-otway

6 found brute force https://muckrack.com/joseph-zucker

7 found brute force https://muckrack.com/simon-duke

8 found brute force https://muckrack.com/maren-estrada

9 found brute force https://muckrack.com/dave-johnson

10 found brute force https://muckrack.com/brett-gibbons

11 found brute force https://muckrack.com/jenna-ciccotelli

12 found brute force https://muckrack.com/kipp-jones

13 found brute force https://muckrack.com/rianne-addo

14 found brute force https://muckrack.com/scott-polacek

15 found brute force https://muckrack.com/clark-schultz

16 found brute force https://muckrack.com/lewis-winter

17 found brute force https://muckrack.com/tom-bull

18 found b

146 found brute force https://muckrack.com/will-twigger

147 found brute force https://muckrack.com/jack-wright

148 found brute force https://muckrack.com/joe-tansey

149 found brute force https://muckrack.com/daniel-murphy

150 found brute force https://muckrack.com/joe-bray

151 found brute force https://muckrack.com/sophie-bateman

152 found brute force https://muckrack.com/chris-kitching

153 found brute force https://muckrack.com/joshua-smith

154 found brute force https://muckrack.com/jenn-gidman

155 found brute force https://muckrack.com/michelle-marshall

156 found brute force https://muckrack.com/rebecca-davison

157 found brute force https://muckrack.com/verity-sulway

158 found brute force https://muckrack.com/kristopher-knox

159 found brute force https://muckrack.com/ben-hooper

160 found brute force https://muckrack.com/rachel-russell

161 found brute force https://muckrack.com/holly-fleet

162 found brute force https://muckrack.com/paul-kasabian

163 found brute force 

288 found brute force https://muckrack.com/emily-hodgkin

289 found brute force https://muckrack.com/joe-pinkstone

290 found brute force https://muckrack.com/jonathan-adams

291 found brute force https://muckrack.com/emer-scully

292 found brute force https://muckrack.com/erin-coates

293 found brute force https://muckrack.com/luke-kenton

294 found brute force https://muckrack.com/sophie-haslett

295 found brute force https://muckrack.com/josh-challies

296 found brute force https://muckrack.com/peter-sblendorio

297 found brute force https://muckrack.com/abbie-llewelyn

298 found brute force https://muckrack.com/tom-davidson

299 found brute force https://muckrack.com/kurt-zindulka

300 found brute force https://muckrack.com/geoff-earle

301 found brute force https://muckrack.com/monica-greep

302 found brute force https://muckrack.com/julie-delahaye

303 found tfidf https://muckrack.com/andy-medici

304 found brute force https://muckrack.com/jessica-mcbride

305 found brute force h

432 found brute force https://muckrack.com/john-binder

433 found brute force https://muckrack.com/monique-friedlander

434 found brute force https://muckrack.com/dade-hayes

435 found brute force https://muckrack.com/matthew-dresch

436 found brute force https://muckrack.com/marlene-lenthang

437 found brute force https://muckrack.com/mary-kekatos

438 found brute force https://muckrack.com/thomas-bristow

439 found brute force https://muckrack.com/connor-oneill

440 found brute force https://muckrack.com/paige-holland

441 found brute force https://muckrack.com/adam-jones

442 found brute force https://muckrack.com/glen-williams

443 found brute force https://muckrack.com/catherine-murphy

444 found brute force https://muckrack.com/breitbart-london

445 found brute force https://muckrack.com/john-hayward

446 found brute force https://muckrack.com/kelly-ashmore

447 found brute force https://muckrack.com/warren-muggleton

448 found brute force https://muckrack.com/cortney-obrien

449

575 found brute force https://muckrack.com/jack-montgomery

576 found brute force https://muckrack.com/david-ng

577 found brute force https://muckrack.com/ross-ibbetson

578 found brute force https://muckrack.com/jem-aswad

579 found brute force https://muckrack.com/daniel-morrow

580 found brute force https://muckrack.com/dan-carden

581 found brute force https://muckrack.com/jane-herz

582 found brute force https://muckrack.com/emily-scrivener

583 found brute force https://muckrack.com/stephen-johnson

584 found brute force https://muckrack.com/tita-smith

585 found brute force https://muckrack.com/frances-martel

586 found brute force https://muckrack.com/ted-johnson

587 found brute force https://muckrack.com/bridie-pearson-jones

588 found brute force https://muckrack.com/brooks-hays

589 found brute force https://muckrack.com/vandana-singh

590 found brute force https://muckrack.com/katie-harris

591 found brute force https://muckrack.com/edgar-thompson

592 found brute force h

...

2696 found brute force https://muckrack.com/shashank-nayar

2697 found brute force https://muckrack.com/ruki-sayid

2698 found brute force https://muckrack.com/cormac-okeeffe

2699 found brute force https://muckrack.com/sasha-lekach

2700 found brute force https://muckrack.com/brian-gilmartin

2701 found brute force https://muckrack.com/tim-reynolds

2702 found brute force https://muckrack.com/tim-schwartz

2703 found brute force https://muckrack.com/dan-gentile

2704 found tfidf https://muckrack.com/fergushunter

2705 found brute force https://muckrack.com/malibongwe-dayimani

2706 found brute force https://muckrack.com/alex-golden

2707 found brute force https://muckrack.com/laurence-hammack

2708 found brute force https://muckrack.com/matthew-daly

2709 found brute force https://muckrack.com/seanna-adcox

2710 found brute force https://muckrack.com/paul-rees

2711 found brute force https://muckrack.com/dave-skretta

2712 found tfidf https://muckrack.com/paul-newman-2

2713 found bru

2838 found brute force https://muckrack.com/arriana-mclymore

2839 found brute force https://muckrack.com/chris-doyle

2840 found brute force https://muckrack.com/jabari-young

2841 found brute force https://muckrack.com/sarah-mitroff

2842 found brute force https://muckrack.com/judy-bergeron

2843 found brute force https://muckrack.com/joe-harris

2844 found brute force https://muckrack.com/john-simerman

2845 found brute force https://muckrack.com/mark-long

2846 found brute force https://muckrack.com/mark-long

2847 found brute force https://muckrack.com/aparajita-saxena

2848 found brute force https://muckrack.com/matt-binder

2849 found brute force https://muckrack.com/tom-brueggemann

2850 found brute force https://muckrack.com/james-picerno

2851 found brute force https://muckrack.com/barry-collins

2852 found tfidf https://muckrack.com/rjfoley

2853 found brute force https://muckrack.com/vivek-deshpande

2854 found brute force https://muckrack.com/james-manso

2855 found brute 

2979 found tfidf https://muckrack.com/isabelhardman

2980 found brute force https://muckrack.com/joe-mclean

2981 found brute force https://muckrack.com/em-holter

2982 found brute force https://muckrack.com/jane-hamilton

2983 found brute force https://muckrack.com/abhinav-rajput

2984 found brute force https://muckrack.com/abhinav-kaul

2985 found brute force https://muckrack.com/abdel-jimenez

2986 found brute force https://muckrack.com/emily-andrews

2987 found brute force https://muckrack.com/james-hurley

2988 found brute force https://muckrack.com/ethan-millman

2989 found brute force https://muckrack.com/eric-kohn

2990 found brute force https://muckrack.com/estelle-shirbon

2991 found brute force https://muckrack.com/justin-ong

2992 found brute force https://muckrack.com/joanna-partridge

2993 found brute force https://muckrack.com/tristan-greene

2994 found brute force https://muckrack.com/david-hambling

2995 found brute force https://muckrack.com/deborah-horn

2996 found b

3120 found brute force https://muckrack.com/emily-gosden

3121 found brute force https://muckrack.com/ananya-bhattacharya

3122 found brute force https://muckrack.com/ishita-ayan-dutt

3123 found brute force https://muckrack.com/wahyudi-soeriaatmadja

3124 found brute force https://muckrack.com/emma-gritt

3125 found brute force https://muckrack.com/sarah-brazendale

3126 found tfidf https://muckrack.com/jacob-carah

3127 found brute force https://muckrack.com/aby-sam-thomas

3128 found brute force https://muckrack.com/sarah-hodgson

3129 found brute force https://muckrack.com/eoin-english

3130 found brute force https://muckrack.com/peter-fitzsimons

3131 found brute force https://muckrack.com/aaron-sheldrick

3132 found brute force https://muckrack.com/sarah-lumley

3133 found brute force https://muckrack.com/georgia-diebelius

3134 found brute force https://muckrack.com/stephanie-kelly

3135 found brute force https://muckrack.com/sarah-obrien

3136 found brute force https://muckrack

3261 found brute force https://muckrack.com/kirsty-needham

3262 found brute force https://muckrack.com/mike-jones

3263 found brute force https://muckrack.com/lameez-omarjee

3264 found brute force https://muckrack.com/roisin-kelly

3265 found brute force https://muckrack.com/callum-keown

3266 found brute force https://muckrack.com/ashna-butani

3267 found brute force https://muckrack.com/daniel-jones

3268 found brute force https://muckrack.com/ashley-harding

3269 found brute force https://muckrack.com/kristen-lee

3270 found brute force https://muckrack.com/greg-wood

3271 found brute force https://muckrack.com/heather-osbourne

3272 found brute force https://muckrack.com/nandagopal-rajan

3273 found brute force https://muckrack.com/danyelle-khmara

3274 found brute force https://muckrack.com/christopher-tan

3275 found brute force https://muckrack.com/sindhu-sundar

3276 found brute force https://muckrack.com/cheryl-teh

3277 found tfidf https://muckrack.com/marissarothkopf

3278

In [10]:
data.to_pickle('../Data/all_muckrack_links.pkl')

In [11]:
data

,author,article_count,request_url,redirect_url,method,confidence
0,Neil Shaw,523,https://muckrack.com/neil-shaw,https://muckrack.com/neil-shaw-1,brute force,-
1,James Rodger,512,https://muckrack.com/james-rodger,https://muckrack.com/james-rodger,brute force,-
2,Jack Davis,251,https://muckrack.com/jack-davis,https://muckrack.com/jack-davis,brute force,-
3,Adam Wells,223,https://muckrack.com/adam-wells,https://muckrack.com/adam-wells,brute force,-
4,Sophie McCoid,218,https://muckrack.com/sophie-mccoid,https://muckrack.com/sophie-mccoid,brute force,-
...,...,...,...,...,...,...
3833,Levi Sumagaysay,10,https://muckrack.com/levi-sumagaysay,https://muckrack.com/levi-sumagaysay,brute force,-
3834,Ben Ashford,10,https://muckrack.com/ben-ashford,https://muckrack.com/ben-ashford,brute force,-
3835,Craig Brown,10,https://muckrack.com/craig-brown,https://muckrack.com/craig-brown,brute force,-
3836,Courtney Connley,10,https://muckrack.com/courtney-connley,https://muckrack.com/courtney-connley,brute force,-


In [16]:
data[data.request_url != data.redirect_url]

,author,article_count,request_url,redirect_url,method,confidence
0,Neil Shaw,523,https://muckrack.com/neil-shaw,https://muckrack.com/neil-shaw-1,brute force,-
7,Simon Duke,214,https://muckrack.com/simon-duke,https://muckrack.com/simonduketimes,brute force,-
13,Rianne Addo,168,https://muckrack.com/rianne-addo,https://muckrack.com/rianneaddo,brute force,-
15,SA Editor Clark Schultz,161,https://muckrack.com/clark-schultz,https://muckrack.com/clarkschultz,brute force,-
16,Lewis Winter,160,https://muckrack.com/lewis-winter,https://muckrack.com/lewisawinter,brute force,-
...,...,...,...,...,...,...
3802,Colin Kruger,10,https://muckrack.com/colin-kruger,https://muckrack.com/colinjkruger,brute force,-
3804,Rafael Olmeda,10,https://muckrack.com/rafael-olmeda,https://muckrack.com/rolmeda,brute force,-
3814,Rachel Koning Beals,10,https://muckrack.com/rachel-koning-beals,https://muckrack.com/rachelkbeals,brute force,-
3817,Corey Jones Tulsa World,10,https://muckrack.com/corey-jones,https://muckrack.com/coreyhjones_,brute force,-


In [ ]:
urlopen(req).url

In [ ]:
data[:100][data.request_url != data.redirect_url]